In [1]:
import glob
import pandas as pd
import numpy as np

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
from keras.layers import LSTM, Dense, Conv1D, TimeDistributed, Flatten, Activation, Dropout, Bidirectional
from keras.callbacks import History, TensorBoard, Callback
import keras.initializers as KI
from keras.layers import BatchNormalization
from sklearn.metrics import f1_score, precision_score, recall_score

from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import warnings
#warnings.filterwarnings("ignore")
from sklearn.model_selection import LeaveOneGroupOut
from tensorflow.keras.optimizers import Adam

2023-07-13 17:10:38.877456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.experimental.set_visible_devices([], 'GPU')

In [2]:

#merge the gaze labelled data for each participant and document
gaze_path = "../../data/working/gaze_labelled/"

gaze_list = [pd.read_csv(data, index_col=0) for data in sorted(glob.glob(gaze_path + "/*/*"))]
gaze_data = pd.concat(gaze_list, ignore_index=True, sort=False)

In [4]:
gaze_data.columns
features = ['left_gaze_x', 'left_gaze_y',
       'left_position_x', 'left_position_y', 'left_position_z', 'left_pupil',
       'right_gaze_x', 'right_gaze_y', 'right_position_x', 'right_position_y',
       'right_position_z', 'right_pupil','understand','interest']


gaze_data = gaze_data[(gaze_data['document'] != 'd20') & (gaze_data['document'] != 'd06')]
participants = gaze_data.participant.unique()
documents = gaze_data.document.unique()


In [8]:
gaze_data['interest'] = gaze_data['interest'].replace([1,2,3,4], [0,1,2,3])
gaze_data['understand'] = gaze_data['understand'].replace([1,2,3,4], [0,1,2,3])


In [9]:
# Extract the features and labels from the data
X = gaze_data.drop(['#timestamp', 'smi_timestamp','participant', 'document', 'interest'], axis=1).values
y = gaze_data['interest'].values
part = gaze_data.participant.values
num_participants = np.unique(participants).shape[0]

In [10]:
samp_freq = 60
window_size = samp_freq * 10
step_size = samp_freq * 5

logo = LeaveOneGroupOut()
acc_per_fold = []

# Iterate over each train-test split based on participants
for train_index, test_index in logo.split(X, y, groups=part):
    # Split the data into train and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Apply sliding window to the training data
    X_train_windows = []
    y_train_windows = []
    for i in range(0, len(X_train) - window_size, step_size):
        X_train_windows.append(X_train[i:i+window_size])
        y_train_windows.append(stats.mode(y_train[i:i+window_size])[0][0])  
    
    X_train_windows = np.array(X_train_windows)
    y_train_windows = np.array(y_train_windows)


    # Apply sliding window to the testing data
    X_test_windows = []
    y_test_windows = []
    for i in range(0, len(X_test) - window_size, step_size):
        X_test_windows.append(X_test[i:i+window_size])
        y_test_windows.append(stats.mode(y_test[i:i+window_size])[0][0])  # Label corresponds to the mode of the window

    X_test_windows = np.array(X_test_windows)
    y_test_windows = np.array(y_test_windows)


    # Normalize the data
    X_train_windows = (X_train_windows - np.mean(X_train_windows)) / np.std(X_train_windows)
    X_test_windows = (X_test_windows - np.mean(X_train_windows)) / np.std(X_train_windows)

    #X_train_windows = X_train_windows.reshape(X_train_windows.shape[0], X_train_windows.shape[1], X_train_windows.shape[2], 1)
    #X_test_windows = X_test_windows.reshape(X_test_windows.shape[0], X_test_windows.shape[1], X_test_windows.shape[2], 1)

    # Define and compile the 1D CNN model
    model = Sequential()
    model.add(Conv1D(100, 5, activation='relu', input_shape=(X_train_windows.shape[1], X_train_windows.shape[2])))
    model.add(Conv1D(100, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(50, 2, activation='relu'))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with sliding window data
    model.fit(X_train_windows, y_train_windows, epochs=10, batch_size=256,verbose=2, validation_split=0.3)# Define the leave-one-participant-out cross-validation strategy

    scores = model.evaluate(X_test_windows, y_test_windows, verbose=2)

# Store the evaluation metrics for this fold
    acc_per_fold.append(scores[1])

 # Compute the average evaluation metrics
avg_acc = np.mean(acc_per_fold)

# # Print the average evaluation metrics
print(f'Average Accuracy: {avg_acc}') 

/var/folders/pk/h_04kxmd1tjfkcy0w0j4b5xc0000gn/T/ipykernel_63178/3661363449.py:19: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_train_windows.append(stats.mode(y_train[i:i+window_size])[0][0])
/var/folders/pk/h_04kxmd1tjfkcy0w0j4b5xc0000gn/T/ipykernel_63178/3661363449.py:30: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accept

Epoch 1/10


2023-07-13 18:11:11.840319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-13 18:11:11.922334: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2023-07-13 18:11:14.490332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


23/23 - 4s - loss: 2.4599 - accuracy: 0.3586 - val_loss: 1.2978 - val_accuracy: 0.3535 - 4s/epoch - 174ms/step
Epoch 2/10
23/23 - 2s - loss: 1.3239 - accuracy: 0.3641 - val_loss: 1.1482 - val_accuracy: 0.3659 - 2s/epoch - 92ms/step
Epoch 3/10
23/23 - 2s - loss: 1.1553 - accuracy: 0.3831 - val_loss: 1.0916 - val_accuracy: 0.4133 - 2s/epoch - 93ms/step
Epoch 4/10
23/23 - 2s - loss: 1.2721 - accuracy: 0.3709 - val_loss: 1.7298 - val_accuracy: 0.3790 - 2s/epoch - 93ms/step
Epoch 5/10
23/23 - 2s - loss: 4.5706 - accuracy: 0.3605 - val_loss: 7.2125 - val_accuracy: 0.1742 - 2s/epoch - 93ms/step
Epoch 6/10
23/23 - 2s - loss: 2.5367 - accuracy: 0.3637 - val_loss: 1.8358 - val_accuracy: 0.2164 - 2s/epoch - 92ms/step
Epoch 7/10
23/23 - 2s - loss: 1.3658 - accuracy: 0.3538 - val_loss: 1.4875 - val_accuracy: 0.4305 - 2s/epoch - 92ms/step
Epoch 8/10
23/23 - 2s - loss: 1.2070 - accuracy: 0.4111 - val_loss: 1.1979 - val_accuracy: 0.3392 - 2s/epoch - 93ms/step
Epoch 9/10
23/23 - 2s - loss: 1.4192 - acc

2023-07-13 18:11:37.798973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-13 18:11:37.887780: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_10.
2023-07-13 18:11:40.466429: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


22/22 - 4s - loss: 1.4483 - accuracy: 0.3713 - val_loss: 1.7506 - val_accuracy: 0.4969 - 4s/epoch - 184ms/step
Epoch 2/10
22/22 - 2s - loss: 1.4358 - accuracy: 0.3731 - val_loss: 1.2811 - val_accuracy: 0.2230 - 2s/epoch - 93ms/step
Epoch 3/10
22/22 - 2s - loss: 1.1262 - accuracy: 0.3955 - val_loss: 1.1563 - val_accuracy: 0.3372 - 2s/epoch - 92ms/step
Epoch 4/10


KeyboardInterrupt: 